In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
#import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline

from tensorflow.keras.losses import Loss

2024-02-25 13:21:05.581035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 13:21:05.581123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 13:21:05.703783: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def add_layer(tensor):
    return tensor[0] + tensor[1]

def mul_layer(tensor):
    return tensor[0] * tensor[1]

def div_layer(tensor):
    return tensor[0] / tensor[1]

def sub_layer(tensor):
    return tensor[0] - tensor[1]

def neg_layer(tensor):
    return -tensor

def cos_layer(tensor):
    return tf.math.cos(tensor)

def sin_layer(tensor):
    return tf.math.sin(tensor)

In [3]:
# feature extraction from left image

left_img = Input(shape = (150,150,3), name="left_image")

# feature extraction from right image
right_img = Input(shape = (150,150,3), name="right_image")

left_phi_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=left_img, input_shape=(150,150,3))
for layer in left_phi_model.layers:
  layer._name = layer._name+"f1_base"

left_phi_features = left_phi_model.output
left_flat = Flatten(name='left-phi-flattened')(left_phi_features)

right_phi_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=right_img, input_shape=(150,150,3))
for layer in right_phi_model.layers:
  layer._name = layer._name+"f2_base"

right_phi_features = right_phi_model.output
right_flat = Flatten(name='right-phi-flattened')(right_phi_features)

concat = concatenate([left_flat, right_flat])

# fx
dense_1 = Dense(120, activation = 'relu')(concat)
dense_2 = Dense(84, activation = 'relu')(dense_1)
pred_fx = Dense(1, name='fx')(dense_2)

# fy
dense_3 = Dense(120, activation = 'relu')(concat)
dense_4 = Dense(84, activation = 'relu')(dense_3)
pred_fy = Dense(1, name='fy')(dense_4)

# u0
dense_5 = Dense(120, activation = 'relu')(concat)
dense_6 = Dense(84, activation = 'relu')(dense_5)
pred_u0 = Dense(1, name='u0')(dense_6)

# v0
dense_7 = Dense(120, activation = 'relu')(concat)
dense_8 = Dense(84, activation = 'relu')(dense_7)
pred_v0 = Dense(1, name='v0')(dense_8)

# baseline
dense_9 = Dense(120, activation = 'relu')(concat)
dense_10 = Dense(84, activation = 'relu')(dense_9)
pred_baseline = Dense(1, name='baseline')(dense_10)

# tx
dense_11 = Dense(120, activation = 'relu')(concat)
dense_12 = Dense(84, activation = 'relu')(dense_11)
pred_x = Dense(1, name='x')(dense_12)

# ty
dense_13 = Dense(120, activation = 'relu')(concat)
dense_14 = Dense(84, activation = 'relu')(dense_13)
pred_y = Dense(1, name='y')(dense_14)

# tz
dense_15 = Dense(120, activation = 'relu')(concat)
dense_16 = Dense(84, activation = 'relu')(dense_15)
pred_z = Dense(1, name='z')(dense_16)

# pitch
dense_17 = Dense(120, activation = 'relu')(concat)
dense_18 = Dense(84, activation = 'relu')(dense_17)
pred_pitch = Dense(1, name='pitch')(dense_18)

# u
dense_19 = Dense(120, activation = 'relu')(concat)
dense_20 = Dense(84, activation = 'relu')(dense_19)
pred_u = Dense(1, name='u')(dense_20)

# v
dense_21 = Dense(120, activation = 'relu')(concat)
dense_22 = Dense(84, activation = 'relu')(dense_21)
pred_v = Dense(1, name='v')(dense_22)

# disparity
dense_23 = Dense(120, activation = 'relu')(concat)
dense_24 = Dense(84, activation = 'relu')(dense_23)
pred_disparity = Dense(1, name='disparity')(dense_24)

# w_xworld: alpha for X 3D point
dense_31 = Dense(120, activation = 'relu')(concat)
dense_32 = Dense(84, activation = 'relu')(dense_31)
w_xworld = Dense(1, name='w_xworld', activation = 'sigmoid')(dense_32)

# w_yworld: alpha for Y 3D point
dense_33 = Dense(120, activation = 'relu')(concat)
dense_34 = Dense(84, activation = 'relu')(dense_33)
w_yworld = Dense(1, name='w_yworld', activation = 'sigmoid')(dense_34)

# w_zworld: alpha for Z 3D point
dense_35 = Dense(120, activation = 'relu')(concat)
dense_36 = Dense(84, activation = 'relu')(dense_35)
w_zworld = Dense(1, name='w_zworld', activation = 'sigmoid')(dense_36)

#vp+_+ is for vanishing points

# alpha_vp1_1
dense_vp1_1_1 = Dense(120, activation = 'relu')(concat)
dense_vp1_1_2 = Dense(84, activation = 'relu')(dense_vp1_1_1)
alpha_vp1_1 = Dense(1, name='alpha_vp1_1', activation = 'sigmoid')(dense_vp1_1_2)

# alpha_vp1_2
dense_vp1_2_1 = Dense(120, activation = 'relu')(concat)
dense_vp1_2_2 = Dense(84, activation = 'relu')(dense_vp1_2_1)
alpha_vp1_2 = Dense(1, name='alpha_vp1_2', activation = 'sigmoid')(dense_vp1_2_2)

# alpha_vp2_1
dense_vp2_1_1 = Dense(120, activation = 'relu')(concat)
dense_vp2_1_2 = Dense(84, activation = 'relu')(dense_vp2_1_1)
alpha_vp2_1 = Dense(1, name='alpha_vp2_1', activation = 'sigmoid')(dense_vp2_1_2)

# alpha_vp2_2
dense_vp2_2_1 = Dense(120, activation = 'relu')(concat)
dense_vp2_2_2 = Dense(84, activation = 'relu')(dense_vp2_2_1)
alpha_vp2_2 = Dense(1, name='alpha_vp2_2', activation = 'sigmoid')(dense_vp2_2_2)

#wc+_+ is for word center

# alpha_wc1_1
dense_wc1_1_1 = Dense(120, activation = 'relu')(concat)
dense_wc1_1_2 = Dense(84, activation = 'relu')(dense_wc1_1_1)
alpha_wc1_1 = Dense(1, name='alpha_wc1_1', activation = 'sigmoid')(dense_wc1_1_2)

# alpha_wc1_2
dense_wc1_2_1 = Dense(120, activation = 'relu')(concat)
dense_wc1_2_2 = Dense(84, activation = 'relu')(dense_wc1_2_1)
alpha_wc1_2 = Dense(1, name='alpha_wc1_2', activation = 'sigmoid')(dense_wc1_2_2)

# alpha_con: alpha for all the constraints
dense_con1 = Dense(120, activation = 'relu')(concat)
dense_con2 = Dense(84, activation = 'relu')(dense_con1)
alpha_con = Dense(1, name='alpha_con', activation = 'sigmoid')(dense_con2)

# alpha_3d: alpha for all the 3d points
dense_3d1 = Dense(120, activation = 'relu')(concat)
dense_3d2 = Dense(84, activation = 'relu')(dense_3d1)
alpha_3d = Dense(1, name='alpha_3d', activation = 'sigmoid')(dense_3d2)

# For 3D point:

# xCam = (self.intrinsic.fx * self.extrinsic.baseline) / disparity
mul_1 = Lambda(mul_layer)([pred_fx, pred_baseline])
xCam = Lambda(div_layer)([mul_1, pred_disparity])

# yCam = - (xCam / self.intrinsic.fx) * (u - self.intrinsic.u0)
div_1 = Lambda(div_layer)([xCam, pred_fx])
sub_1 = Lambda(sub_layer)([pred_u, pred_u0])
yCam = Lambda(mul_layer)([Lambda(neg_layer)(div_1), sub_1])

# zCam = (xCam / self.intrinsic.fy) * (self.intrinsic.v0 - v)
div_2 = Lambda(div_layer)([xCam, pred_fy])
sub_2 = Lambda(sub_layer)([pred_v0, pred_v])
zCam = Lambda(mul_layer)([div_2, sub_2])

# Y = yCam + self.extrinsic.y
pred_yWorld = Lambda(add_layer)([yCam, pred_y])
pred_yWorld = Lambda(mul_layer, name='yWorld')([pred_yWorld, w_yworld]) * alpha_3d

# X = xCam * math.cos(self.extrinsic.pitch) + zCam * math.sin(self.extrinsic.pitch) + self.extrinsic.x
mul_2 = Lambda(mul_layer)([xCam, Lambda(cos_layer)(pred_pitch)])
mul_3 = Lambda(mul_layer)([zCam, Lambda(sin_layer)(pred_pitch)])
add_1 = Lambda(add_layer)([mul_2, mul_3])
pred_xWorld = Lambda(add_layer)([add_1, pred_x])
pred_xWorld = Lambda(mul_layer, name='xWorld')([pred_xWorld, w_xworld]) * alpha_3d

# Z = - xCam * math.sin(self.extrinsic.pitch) + zCam * math.cos(self.extrinsic.pitch) + self.extrinsic.z
mul_4 = Lambda(mul_layer)([Lambda(neg_layer)(xCam), Lambda(sin_layer)(pred_pitch)])
mul_5 = Lambda(mul_layer)([zCam, Lambda(cos_layer)(pred_pitch)])
add_2 = Lambda(add_layer)([mul_4, mul_5])
pred_zWorld = Lambda(add_layer)([add_2, pred_z])
pred_zWorld = Lambda(mul_layer, name='zWorld')([pred_zWorld, w_zworld]) * alpha_3d

# For Constraints

dim = tf.shape(pred_fx)[0]

k_00 = pred_fx
k_02 = pred_u0
k_11 = pred_fy
k_12 = pred_v0

# k = intrinsic matrix

k = tf.stack([
    tf.stack([k_00, tf.zeros((dim, 1), dtype=tf.float32), k_02], axis=1),
    tf.stack([tf.zeros((dim, 1), dtype=tf.float32), k_11, k_12], axis=1),
    tf.stack([tf.zeros((dim, 1), dtype=tf.float32), tf.zeros((dim, 1), dtype=tf.float32), tf.ones((dim, 1), dtype=tf.float32)], axis=1)
], axis=2)

k = tf.reshape(tf.transpose(tf.reshape(k, (dim, 3, 3)), perm=[0, 2, 1]), (dim, 3, 3))


r_pitch = tf.stack([
    tf.stack([tf.cos(pred_pitch), tf.zeros(shape=(dim, 1), dtype=tf.float32), tf.sin(pred_pitch)], axis=1),
    tf.stack([tf.zeros(shape=(dim, 1), dtype=tf.float32), tf.ones(shape=(dim, 1), dtype=tf.float32), tf.zeros(shape=(dim, 1), dtype=tf.float32)], axis=1),
    tf.stack([-tf.sin(pred_pitch), tf.zeros(shape=(dim, 1), dtype=tf.float32), tf.cos(pred_pitch)], axis=1)
], axis=2)


r_pitch = tf.reshape(tf.transpose(tf.reshape(r_pitch, (dim, 3, 3)), perm=[0, 2, 1]), (dim, 3, 3))

# t = translation
t = tf.stack([pred_x, pred_y, pred_z], axis=1)

# r = rotation + t
r = tf.concat([r_pitch, t], axis=2)

# projection matrix
p = tf.linalg.matmul(k, r) + 1

vanish_point_1 = tf.gather(p, [0, 2, 3], axis=2)

vp1_1 = Lambda(mul_layer, name='vp1_1')([
    alpha_vp1_1,
    vanish_point_1[:, :, 0][:, 0:1] / ((vanish_point_1[:, :, 0][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con

vp1_2 = Lambda(mul_layer, name='vp1_2')([
    alpha_vp1_2,
    vanish_point_1[:, :, 0][:, 1:2] / ((vanish_point_1[:, :, 0][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con

vp2_1 = Lambda(mul_layer, name='vp2_1')([
    alpha_vp2_1,
    vanish_point_1[:, :, 1][:, 0:1] / ((vanish_point_1[:, :, 1][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con

vp2_2 = Lambda(mul_layer, name='vp2_2')([
    alpha_vp2_2,
    vanish_point_1[:, :, 1][:, 1:2] / ((vanish_point_1[:, :, 1][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con

wc1_1 = Lambda(mul_layer, name='wc1_1')([
    alpha_wc1_1,
    vanish_point_1[:, :, 2][:, 1:2] / ((vanish_point_1[:, :, 2][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con

wc1_2 = Lambda(mul_layer, name='wc1_2')([
    alpha_wc1_2,
    vanish_point_1[:, :, 2][:, 1:2] / ((vanish_point_1[:, :, 2][:, 2:3] + 0.00001) + 0.00001)
]) * alpha_con


# Row Constraints

# 1) Product of Rows
row_prod = r_pitch[:, 0] * r_pitch[:, 2]
row_product = tf.reduce_sum(row_prod, axis=1, name="rot_row_product")

# 2) Determinant
determinant = tf.linalg.det(r_pitch, name='determinant')

# 3) Identity

pred_pitch_transposed = tf.transpose(r_pitch, perm=(0, 2, 1))
element_wise_mul = r_pitch * pred_pitch_transposed
sum_axis_1 = tf.reduce_sum(element_wise_mul, axis=1)
rot_identity = tf.reduce_sum(sum_axis_1, axis=1, name='rot_identity')

# create model
model = Model(inputs=[left_img, right_img], outputs=[pred_fx, pred_fy, pred_u0, pred_v0, pred_baseline, pred_disparity, pred_x, pred_y, pred_z, pred_pitch, pred_xWorld,pred_yWorld,pred_zWorld, vp1_1, vp1_2, vp2_1, vp2_2, wc1_1, wc1_2, row_product, determinant, rot_identity])

# set output types
target1 = tf.placeholder(dtype='float32', shape=(1,1))
target2 = tf.placeholder(dtype='float32', shape=(1,1))
target3 = tf.placeholder(dtype='float32', shape=(1,1))
target4 = tf.placeholder(dtype='float32', shape=(1,1))
target5 = tf.placeholder(dtype='float32', shape=(1,1))
target6 = tf.placeholder(dtype='float32', shape=(1,1))
target7 = tf.placeholder(dtype='float32', shape=(1,1))
target8 = tf.placeholder(dtype='float32', shape=(1,1))
target9 = tf.placeholder(dtype='float32', shape=(1,1))
target10 = tf.placeholder(dtype='float32', shape=(1,1))
target11 = tf.placeholder(dtype='float32', shape=(1,1))
target12 = tf.placeholder(dtype='float32', shape=(1,1))
target13 = tf.placeholder(dtype='float32', shape=(1,1))

t_vp1_1 = tf.placeholder(dtype='float32', shape=(1,1))
t_vp1_2 = tf.placeholder(dtype='float32', shape=(1,1))
t_vp2_1 = tf.placeholder(dtype='float32', shape=(1,1))
t_vp2_2 = tf.placeholder(dtype='float32', shape=(1,1))
t_wc1_1 = tf.placeholder(dtype='float32', shape=(1,1))
t_wc1_2 = tf.placeholder(dtype='float32', shape=(1,1))

row = tf.placeholder(dtype='float32', shape=(1,1))
det = tf.placeholder(dtype='float32', shape=(1,1))
row_idt = tf.placeholder(dtype='float32', shape=(1,1))


learning_rate = 0.001

# compile model
model.compile(loss="mae", target_tensors=[target1, target2, target3, target4, target5, target6, target7, target8, target9, target10, target11, target12, target13, t_vp1_1, t_vp1_2, t_vp2_1, t_vp2_2, t_wc1_1, t_wc1_2, row, det, row_idt], optimizer=optimizers.legacy.Adam(lr=learning_rate));
#plot_model(model, to_file='model.png')

87910968/87910968 [==============================] - 0s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
import numpy as np
data_path = 'your pth/'
Fx = np.load(data_path+"fx.npy", mmap_mode='r')
Fy = np.load(data_path+"fy.npy", mmap_mode='r')
U0 = np.load(data_path+"U0.npy", mmap_mode='r')
V0 = np.load(data_path+"V0.npy", mmap_mode='r')
Disparity = np.load(data_path+"disparity.npy", mmap_mode='r')
Tx = np.load(data_path+"x.npy", mmap_mode='r')
Ty = np.load(data_path+"y.npy", mmap_mode='r') 
Tz = np.load(data_path+"z.npy", mmap_mode='r')
Baseline = np.load(data_path+"baseline.npy", mmap_mode='r')
Pitch = np.load(data_path+"pitch.npy", mmap_mode='r')
X = np.load(data_path+"xworld.npy", mmap_mode='r')
Y = np.load(data_path+"yworld.npy", mmap_mode='r')
Z = np.load(data_path+"zworld.npy", mmap_mode='r')
Left_images = np.load(data_path+"left_img.npy", mmap_mode='r')
Right_images = np.load(data_path+"right_img.npy", mmap_mode='r')

In [12]:
import os
from keras.callbacks import TensorBoard, LearningRateScheduler
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
#from utils_regressor_focal_dist import RotNetDataGenerator, angle_error, CustomModelCheckpoint
from keras import optimizers
import numpy as np
import glob, math
from shutil import copyfile
import datetime, random
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session


model.load_weights('UGCL-VP-WC-R.h5')

input_shape = (150, 150, 3)


output = model.predict(
    x=[Left_images[int(len(Left_images)*0.7):], Right_images[int(len(Left_images)*0.7):]],
    batch_size=32,
    verbose=1
)

In [15]:
len(output[0]), int(len(Left_images)*0.7)

(19080, 44520)

In [17]:

error = {}

error["fx"] = 0
error["fy"] = 0
error["u0"] = 0
error["v0"] = 0
error["baseline"] = 0
error["disparity"] = 0
error["x"] = 0
error["y"] = 0
error["z"] = 0
error["pitch"] = 0

key_counter = 0

for i in error.keys():
    
    k = int(len(Left_images)*0.7)

    for j  in range(len(output[0])):
        
        if key_counter == 0: 
    
            predicted_fx = output[key_counter][j][0]
            actual_fx = Fx[k]
    
            error[i] += abs(predicted_fx - actual_fx)
        
        if key_counter == 1: 
    
            predicted_fy = output[key_counter][j][0]
            actual_fy = Fy[k]
    
            error[i] += abs(predicted_fy - actual_fy)
        
        if key_counter == 2: 
    
            predicted_u0 = output[key_counter][j][0]
            actual_u0 = U0[k]
    
            error[i] += abs(predicted_u0 - actual_u0)
        
        if key_counter == 3: 
    
            predicted_v0 = output[key_counter][j][0]
            actual_v0 = V0[k]
    
            error[i] += abs(predicted_v0 - actual_v0)
        
        if key_counter == 4: 
    
            predicted_baseline = output[key_counter][j][0]
            actual_baseline = Baseline[k]
    
            error[i] += abs(predicted_baseline - actual_baseline)
        
        if key_counter == 5: 
    
            predicted_disparity = output[key_counter][j][0]
            actual_disparity = Disparity[k]
    
            error[i] += abs(predicted_disparity - actual_disparity)
        
        if key_counter == 6: 
    
            predicted_tx = output[key_counter][j][0]
            actual_tx = Tx[k]
    
            error[i] += abs(predicted_tx - actual_tx)
        
        if key_counter == 7: 
    
            predicted_ty = output[key_counter][j][0]
            actual_ty = Ty[k]
    
            error[i] += abs(predicted_ty - actual_ty)
        
        if key_counter == 8: 
    
            predicted_tz = output[key_counter][j][0]
            actual_tz = Tz[k]
    
            error[i] += abs(predicted_tz - actual_tz)
        
        if key_counter == 9: 
    
            predicted_pitch = output[key_counter][j][0]
            actual_pitch = Pitch[k]
    
            error[i] += abs(predicted_pitch - actual_pitch)
        
        if key_counter == 10: 
    
            predicted_x = output[key_counter][j][0]
            actual_x = X[k]
    
            error[i] += abs(predicted_x - actual_x)
        
        if key_counter == 11: 
    
            predicted_y = output[key_counter][j][0]
            actual_y = Y[k]
    
            error[i] += abs(predicted_y - actual_y)
        
        if key_counter == 12: 
    
            predicted_z = output[key_counter][j][0]
            actual_z = Z[k]
    
            error[i] += abs(predicted_z - actual_z)
        
        k += 1
    
    
    error[i] /= int(len(Left_images)*0.7)
    
    key_counter += 1

print(error)

{'fx': 0.7491490070003678, 'fy': 0.7733582511243773, 'u0': 0.05935749232179928, 'v0': 0.038121877055069614, 'baseline': 0.0611332172194577, 'disparity': 1.090198556746311, 'x': 0.08577409175035655, 'y': 0.05374187468002219, 'z': 0.054159489964656894, 'pitch': 0.003969053437146964}
